In [1]:
import random
import re
import os

In [2]:
def select_new_type(modifications):
    r = random.uniform(0, 100)
    cumulative = 0
    for new_type, pct in modifications.items():
        cumulative += pct
        if r < cumulative:
            return new_type
    return '1'

In [3]:
def modify_data_file(input_file, output_file, modifications, new_masses=None):
    
    """
    modifications: dict mapping new type (str) to percentage (e.g. {'2': 30, '3': 15, '4': 45})
    new_masses (optional): dict mapping new type (str) to mass (float) for new types (e.g. {'2': 78.2, '3': 104.11, '4': 93.0})
    Assumes original file contains only type 1 atoms.
    """
    
    new_atom_types = max([1] + [int(k) for k in modifications.keys()])
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Split header and body (Atoms section)
    # If new_masses=None, then delete the part of the header with masses
    body_init_idx = next((i for i, line in enumerate(lines) if "Atoms" in line), len(lines))
    if new_masses == None :
        header_end_idx = next((i for i, line in enumerate(lines) if "Masses" in line), len(lines))
    if new_masses == None and header_end_idx<=body_init_idx:
        header = lines[:header_end_idx]
    else :
        header = lines[:body_init_idx]
    body = lines[body_init_idx:]

    # Update "atom types" line in header
    for i, line in enumerate(header):
        if "atom types" in line:
            parts = line.split()
            parts[0] = str(new_atom_types)
            header[i] = ' '.join(parts) + '\n'
            break

    if new_masses != None :
        # Replace Masses block in header
        masses_idx = next((i for i, line in enumerate(header) if "Masses" in line), None)
        if masses_idx is not None:
            masses_block_end = masses_idx + 1
            while masses_block_end < len(header):
                stripped = header[masses_block_end].strip()
                if stripped == "" or re.match(r'^\d+\s+.*', stripped):
                    masses_block_end += 1
                else:
                    break
            mass1 = None
            for j in range(masses_idx + 1, masses_block_end):
                if header[j].strip() != "":
                    parts = header[j].strip().split()
                    if parts[0] == '1':
                        mass1 = parts[1]
                        break
            if mass1 is None:
                mass1 = "0.0"
            new_masses_block = ["Masses\n", "\n"]
            for i in range(1, new_atom_types + 1):
                if i == 1:
                    new_masses_block.append(f"1 {mass1}\n")
                else:
                    mass_val = new_masses.get(str(i), "0.0")
                    new_masses_block.append(f"{i} {mass_val}\n")
            header = header[:masses_idx] + new_masses_block + header[masses_block_end:]

    # Process Atoms section in body
    modified_body = []
    in_atoms_section = False
    for line in body:
        if "Atoms" in line:
            in_atoms_section = True
            modified_body.append(line)
            continue
        if in_atoms_section:
            if any(section in line for section in ["Velocities", "Bonds", "Angles"]):
                in_atoms_section = False
                modified_body.append(line)
                continue
            if line.strip() == "":
                modified_body.append(line)
                continue
            parts = line.strip().split()
            if parts[1] == '1':
                parts[1] = select_new_type(modifications)
            modified_body.append(' '.join(parts) + '\n')
        else:
            modified_body.append(line)

    new_lines = header + modified_body
    with open(output_file, 'w') as f:
        f.writelines(new_lines)
    print(f"Modified file saved as: {output_file}")

In [4]:
!pwd

/home/michele/workflow-hea/ti-al-droplets


In [5]:
!ls

diffusion	      droplet_Ti.data		viscosity
droplet_Al.data       equilibrate-alti.lammps	viscosity-npt
droplet_AlTi.data     log.lammps		viscosity-nve
droplet_AlTi_eq.data  melting-ti.lammps
droplets	      system_preparation.ipynb


In [6]:
input_file = "/home/michele/workflow-hea/ti-al-droplets/droplets/molten_Al.data"
of_folder_root = "/home/michele/workflow-hea/ti-al-droplets/droplets/"

In [7]:
alpha = {
    '000' : 0.0, 
    '020' : 20.0, 
    '040' : 40.0, 
    '060' : 60.0, 
    '080' : 80.0, 
    '100' : 100.0
}

In [8]:
for l, a in alpha.items() :
    os.system("mkdir "+of_folder_root+l)
    output_file = of_folder_root+l+"/molten_AlTi.data"
    modifications = {'2': a}
    modify_data_file(input_file, output_file, modifications)

Modified file saved as: /home/michele/workflow-hea/ti-al-droplets/droplets/000/molten_AlTi.data
Modified file saved as: /home/michele/workflow-hea/ti-al-droplets/droplets/020/molten_AlTi.data
Modified file saved as: /home/michele/workflow-hea/ti-al-droplets/droplets/040/molten_AlTi.data
Modified file saved as: /home/michele/workflow-hea/ti-al-droplets/droplets/060/molten_AlTi.data
Modified file saved as: /home/michele/workflow-hea/ti-al-droplets/droplets/080/molten_AlTi.data
Modified file saved as: /home/michele/workflow-hea/ti-al-droplets/droplets/100/molten_AlTi.data


mkdir: cannot create directory ‘/home/michele/workflow-hea/ti-al-droplets/droplets/000’: File exists
mkdir: cannot create directory ‘/home/michele/workflow-hea/ti-al-droplets/droplets/020’: File exists
mkdir: cannot create directory ‘/home/michele/workflow-hea/ti-al-droplets/droplets/040’: File exists
mkdir: cannot create directory ‘/home/michele/workflow-hea/ti-al-droplets/droplets/060’: File exists
mkdir: cannot create directory ‘/home/michele/workflow-hea/ti-al-droplets/droplets/080’: File exists
mkdir: cannot create directory ‘/home/michele/workflow-hea/ti-al-droplets/droplets/100’: File exists
